# Машинное обучение

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [13]:
import warnings
warnings.filterwarnings('ignore')

В качестве target-переменной возьмём объем продаж - sales. Так как нам нужно предсказать вещественные числа, будем обучать модель линейной регрессии.

Однако, во время промежуточного чекпойнта нам справедливо указали, что модель линейной регрессии плохо справляется с требованиями к сезонности, которая окажет влияние на таргет. 
Поэтому, поскольку мы точно знаем, что нам нужно предсказать данные за август, возьмем данные за все летние месяцы: 

### 1. Подготовим данные

In [14]:
temp = pd.read_csv('temp.csv')
temp.drop(columns='Unnamed: 0', inplace=True)

In [15]:
temp['date'] = pd.to_datetime(temp["date"])

In [16]:
temp['month'] = temp['date'].dt.month

In [17]:
tem = temp[(temp['month'] == 6) | (temp['month'] == 7) | (temp['month'] == 8)]

1.1. Выкинем колонки, которые не будем использовать для обучения: city-state заложены в уникальный store_nbr, country - техническая колонка, которая была нужна для объединения датасетов, от колонок 'national_type' и 'regional_type' зависит, есть ли праздник в определенной локации ('local_type' holiday) - их убираем, никакой дополнительной информации они не принесут. 

In [18]:
tem.drop(columns=['city', 'state', 'country', 'national_type', 'regional_type'], inplace=True)

1.2 Для категориальных переменных используем OHE-кодирование.

In [19]:
tem_ohe = pd.get_dummies(tem, columns=['store_nbr', 'shop_type', 'local_type', 'month'], drop_first= True, dtype='int')

In [20]:
tem_ohe.rename(columns = {'local_type_Holiday':'Holiday', 'local_type_Work Day':'Workday', 'month_7':'July', 
                     'month_8':'Aug'}, inplace = True)

In [21]:
tem_ohe.head(2)

,date,sales,transactions,onpromotion,oilprice,temp,humidity,windspeed(mph),pressure(mbar),precip,...,store_nbr_53,store_nbr_54,shop_type_B,shop_type_C,shop_type_D,shop_type_E,Holiday,Workday,July,Aug
6924,2013-06-01,6180.505,1357,0,92.423333,13,87,3.73,1015,2.9,...,0,0,0,0,1,0,0,1,0,0
6925,2013-06-01,10286.549,2052,0,92.423333,13,87,3.73,1015,2.9,...,0,0,0,0,1,0,0,1,0,0


In [22]:
print(f'до OHE-кодирования: {tem.shape}, после OHE-кодирования: {tem_ohe.shape}')

до OHE-кодирования: (22234, 14), после OHE-кодирования: (22234, 71)


Добавилось 57 новых признаков, в основном за счет магазинов.

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x = tem_ohe.drop(columns=['date', 'sales'])
y = tem_ohe['sales']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

#### 1.3. Приведем все количественные признаки к одному масштабу.  

Все действительные переменные сложим в список real.

In [25]:
real = ['transactions', 'onpromotion', 'oilprice', 'temp', 'humidity',
       'windspeed(mph)', 'pressure(mbar)', 'precip']

In [26]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train[real])
x_test_scaled = scaler.transform(x_test[real])

In [27]:
model = LinearRegression()
model.fit(x_train_scaled, y_train)

y_pred = model.predict(x_test_scaled)
y_train_pred = model.predict(x_train_scaled)

In [28]:
print("Train MAE = %.4f" % metrics.mean_absolute_error(y_train, y_train_pred))
print("Test MAE = %.4f" % metrics.mean_absolute_error(y_test, y_pred))

print(f"Train MSE: { metrics.mean_squared_error(y_train, y_train_pred)}")
print(f"Test MSE: { metrics.mean_squared_error(y_test, y_pred)}")

print(f"Train R2: { metrics.r2_score(y_train, y_train_pred)}")
print(f"Test R2: { metrics.r2_score(y_test, y_pred)}")

Train MAE = 2798.7752
Test MAE = 2717.6549
Train MSE: 15661501.138034316
Test MSE: 14720117.825801818
Train R2: 0.8113132315370712
Test R2: 0.8157733246533494


Значения R2 score для тестовых и тренировочных данных говорят о том, что наша модель описывает данные на 81%. Это довольно хороший результат, но попробуем его ещё немного улучшить. 

#### Lasso регрессия

Посмотрим, поможет ли нам Lasso регрессия:

In [29]:
from sklearn.linear_model import Lasso

In [31]:
from sklearn.model_selection import GridSearchCV

alphas = np.logspace(-2, 3, 20)
searcher = GridSearchCV(
    Lasso(), [{"alpha": alphas}], 
    scoring="neg_mean_squared_error", cv=5
)
searcher.fit(x_train_scaled, y_train)

best_alpha = searcher.best_params_["alpha"]
print("Best alpha = %.4f" % best_alpha)

Best alpha = 0.0336


Наилучший коэффициент альфа вышел очень близким к нулю: Lasso regression при таком параметре мало чем будет отличаться от МНК, используемого в нашей обычной модели => получается, применять Лассо-регрессию не целесообразно.  

**Итог:** на основе ранее созданной таблицы мы построили линейную регрессию, которая достаточно хорошо предсказывает продажи.